In [ ]:
#kaggle script. loading resnet into mxnet, runing xgboost on top

import numpy as np
import dicom
import glob
from matplotlib import pyplot as plt
import os
import cv2
import mxnet as mx
import pandas as pd
from sklearn import cross_validation
import xgboost as xgb


def get_extractor():
    model = mx.model.FeedForward.load('/datadrive/kaggle_ds_bowl_17/resnet-50', 0, ctx=mx.gpu(), numpy_batch_size=1)
    fea_symbol = model.symbol.get_internals()["flatten0_output"]
    feature_extractor = mx.model.FeedForward(ctx=mx.gpu(), symbol=fea_symbol, numpy_batch_size=64,
                                             arg_params=model.arg_params, aux_params=model.aux_params,
                                             allow_extra_params=True)

    return feature_extractor


def get_3d_data(path):
    slices = [dicom.read_file(path + '/' + s) for s in os.listdir(path)]
    slices.sort(key=lambda x: int(x.InstanceNumber))
    return np.stack([s.pixel_array for s in slices])


def get_data_id(path):
    sample_image = get_3d_data(path)
    sample_image[sample_image == -2000] = 0
    # f, plots = plt.subplots(4, 5, sharex='col', sharey='row', figsize=(10, 8))

    batch = []
    cnt = 0
    dx = 40
    ds = 512
    for i in range(0, sample_image.shape[0] - 3, 3):
        tmp = []
        for j in range(3):
            img = sample_image[i + j]
            img = 255.0 / np.amax(img) * img
            img = cv2.equalizeHist(img.astype(np.uint8))
            img = img[dx: ds - dx, dx: ds - dx]
            img = cv2.resize(img, (224, 224))
            tmp.append(img)

        tmp = np.array(tmp)
        batch.append(np.array(tmp))

        # if cnt < 20:
        #     plots[cnt // 5, cnt % 5].axis('off')
        #     plots[cnt // 5, cnt % 5].imshow(np.swapaxes(tmp, 0, 2))
        # cnt += 1

    # plt.show()
    batch = np.array(batch)
    return batch


def calc_features():
    net = get_extractor()
    for folder in glob.glob('/datadrive/kaggle_ds_bowl_17/stage1/*'):
        patient_id=folder.split('/')[-1]
        if '/datadrive/kaggle_ds_bowl_17/resnet_npys/'+patient_id+'.npy' not in glob.glob('/datadrive/kaggle_ds_bowl_17/resnet_npys/*'):
            batch = get_data_id(folder)
            feats = net.predict(batch)
            print(feats.shape)
            np.save('/datadrive/kaggle_ds_bowl_17/resnet_npys/'+patient_id+'.npy', feats)
        else:
            print 'output exists already'


def train_xgboost():
    df = pd.read_csv('/datadrive/kaggle_ds_bowl_17/stage1_labels.csv')
    print(df.head())

    x = np.array([np.mean(np.load('/datadrive/kaggle_ds_bowl_17/resnet_npys/%s.npy' % str(id)), axis=0) for id in df['id'].tolist()])
    y = df['cancer'].as_matrix()

    trn_x, val_x, trn_y, val_y = cross_validation.train_test_split(x, y, random_state=42, stratify=y,
                                                                   test_size=0.20)

    clf = xgb.XGBRegressor(max_depth=10,
                           n_estimators=1500,
                           min_child_weight=9,
                           learning_rate=0.05,
                           nthread=8,
                           subsample=0.80,
                           colsample_bytree=0.80,
                           seed=4242)

    clf.fit(trn_x, trn_y, eval_set=[(val_x, val_y)], verbose=True, eval_metric='logloss', early_stopping_rounds=50)
    return clf


def make_submit():
    clf = train_xgboost()

    df = pd.read_csv('/datadrive/kaggle_ds_bowl_17/stage1_sample_submission.csv')

    x = np.array([np.mean(np.load('/datadrive/kaggle_ds_bowl_17/stage1/%s.npy' % str(id)), axis=0) for id in df['id'].tolist()])

    pred = clf.predict(x)

    df['cancer'] = pred
    df.to_csv('subm1.csv', index=False)
    print(df.head())


if __name__ == '__main__':
    calc_features()
    make_submit()

In [7]:
calc_features()

[Deprecation Warning] mxnet.model.FeedForward has been deprecated. Please use mxnet.mod.Module instead.
[Deprecation Warning] mxnet.model.FeedForward has been deprecated. Please use mxnet.mod.Module instead.


MXNetError: [23:04:00] src/c_api/c_api_ndarray.cc:270: Operator _zeros cannot be run; requires at least one of FCompute<xpu>, NDArrayFunction, FCreateOperator be registered

Stack trace returned 76 entries:
[bt] (0) /home/ubuntu/.local/lib/python2.7/site-packages/mxnet/libmxnet.so(+0xa2c9c) [0x7fef680c6c9c]
[bt] (1) /home/ubuntu/.local/lib/python2.7/site-packages/mxnet/libmxnet.so(MXImperativeInvoke+0x3646) [0x7fef688d2466]
[bt] (2) /usr/lib/x86_64-linux-gnu/libffi.so.6(ffi_call_unix64+0x4c) [0x7fefeb7e8e18]
[bt] (3) /usr/lib/x86_64-linux-gnu/libffi.so.6(ffi_call+0x32a) [0x7fefeb7e887a]
[bt] (4) /usr/lib/python2.7/lib-dynload/_ctypes.x86_64-linux-gnu.so(_ctypes_callproc+0x23f) [0x7fefeb9f73af]
[bt] (5) /usr/lib/python2.7/lib-dynload/_ctypes.x86_64-linux-gnu.so(+0x11603) [0x7fefeb9fc603]
[bt] (6) /usr/bin/python(PyObject_Call+0x43) [0x55bbb8454da3]
[bt] (7) /usr/bin/python(PyEval_EvalFrameEx+0x6092) [0x55bbb846e3f2]
[bt] (8) /usr/bin/python(PyEval_EvalCodeEx+0x255) [0x55bbb8466b85]
[bt]